# AI Inspector v4 - Module Testing

Test notebook for validating v4 modules as they are built.

**Current modules:**
- `config.py` - Configuration
- `models/` - Data models
- `utils/` - Utilities (io, pdf_render, sw_library, context_db)

In [ ]:
# Cell 1: Install dependencies
!pip install -q pymupdf pillow

In [ ]:
# Cell 2: Clone repo (skip if already cloned)
import os
if not os.path.exists('AI-Drawing-Inspector'):
    !git clone https://github.com/skaumbdoallsaws-coder/AI-Drawing-Inspector.git
    %cd AI-Drawing-Inspector
else:
    %cd AI-Drawing-Inspector
    !git pull
    print('Repository updated')

In [ ]:
# Cell 3: Test config module
print('='*50)
print('TEST: config.py')
print('='*50)

from ai_inspector.config import Config, default_config

print(f'Default render_dpi: {default_config.render_dpi}')
print(f'Default output_dir: {default_config.output_dir}')
print(f'VLM model: {default_config.vlm_model_id}')
print(f'Hole tolerance: {default_config.hole_tolerance_inches}"')

# Test custom config
custom = Config(render_dpi=150, output_dir='my_output')
print(f'\nCustom render_dpi: {custom.render_dpi}')
print('\n✓ config.py PASSED')

In [ ]:
# Cell 4: Test models
print('='*50)
print('TEST: models/')
print('='*50)

from ai_inspector.models import (
    PageArtifact,
    ResolvedPartIdentity,
    SwPartEntry,
    PageType,
    PageClassification,
    DrawingClassification,
)
from PIL import Image

# Test PageArtifact
img = Image.new('RGB', (100, 100), color='white')
page = PageArtifact(
    page_index=0, page_number=1, image=img,
    width=100, height=100, dpi=300
)
print(f'PageArtifact: page {page.page_number}, {page.width}x{page.height}')

# Test ResolvedPartIdentity
identity = ResolvedPartIdentity(
    part_number='1008794',
    confidence=1.0,
    source='filename+sw'
)
print(f'ResolvedPartIdentity: {identity.part_number} (conf={identity.confidence})')

# Test PageType enum
print(f'PageType values: {[t.value for t in PageType]}')

print('\n✓ models/ PASSED')

In [ ]:
# Cell 5: Test utils/io.py
print('='*50)
print('TEST: utils/io.py')
print('='*50)

from ai_inspector.utils import load_json_robust
import tempfile
import json

# Create test JSON file
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    json.dump({'test': 'value', 'number': 42}, f)
    temp_path = f.name

data, err = load_json_robust(temp_path)
print(f'Loaded JSON: {data}')
print(f'Error: {err}')

# Test missing file
data, err = load_json_robust('nonexistent.json')
print(f'Missing file error: {err}')

os.unlink(temp_path)
print('\n✓ utils/io.py PASSED')

In [ ]:
# Cell 6: Upload test PDF
print('='*50)
print('UPLOAD: Test PDF for rendering')
print('='*50)

from google.colab import files

print('Upload a PDF drawing to test rendering:')
uploaded = files.upload()

pdf_path = None
for filename in uploaded:
    if filename.lower().endswith('.pdf'):
        pdf_path = filename
        print(f'Using: {pdf_path}')
        break

if not pdf_path:
    print('No PDF uploaded - skipping render test')

In [ ]:
# Cell 7: Test utils/pdf_render.py
print('='*50)
print('TEST: utils/pdf_render.py')
print('='*50)

from ai_inspector.utils import render_pdf
from IPython.display import display

if pdf_path:
    artifacts = render_pdf(pdf_path, dpi=150)  # Lower DPI for speed
    print(f'Rendered {len(artifacts)} page(s)')
    
    for art in artifacts[:3]:  # Show first 3 pages
        print(f'  Page {art.page_number}: {art.width}x{art.height}px')
        print(f'    Direct text: {len(art.direct_text or "")} chars')
        print(f'    needs_ocr: {art.needs_ocr}')
    
    # Display first page thumbnail
    if artifacts:
        thumb = artifacts[0].image.copy()
        thumb.thumbnail((600, 600))
        display(thumb)
    
    print('\n✓ utils/pdf_render.py PASSED')
else:
    print('Skipped - no PDF uploaded')

In [ ]:
# Cell 8: Upload SW JSON library (optional)
print('='*50)
print('UPLOAD: SolidWorks JSON library (optional)')
print('='*50)

import zipfile

SW_JSON_DIR = 'sw_json_library'

if not os.path.exists(SW_JSON_DIR):
    print('Upload sw_json_library.zip (or skip):')
    try:
        uploaded = files.upload()
        for filename in uploaded:
            if filename.endswith('.zip'):
                with zipfile.ZipFile(filename, 'r') as z:
                    z.extractall(SW_JSON_DIR)
                print(f'Extracted to {SW_JSON_DIR}')
                break
    except:
        print('Upload skipped or cancelled')
else:
    print(f'{SW_JSON_DIR} already exists')

In [ ]:
# Cell 9: Test utils/sw_library.py
print('='*50)
print('TEST: utils/sw_library.py')
print('='*50)

from ai_inspector.utils import SwJsonLibrary

library = SwJsonLibrary()

if os.path.exists(SW_JSON_DIR):
    count = library.load_from_directory(SW_JSON_DIR)
    print(f'Loaded {count} parts')
    
    # Show first few
    for entry in library.all_entries[:5]:
        desc = entry.data.get('identity', {}).get('description', 'N/A')[:40]
        print(f'  {entry.part_number}: {desc}')
    
    # Test lookup
    if library.all_entries:
        test_pn = library.all_entries[0].part_number
        result = library.lookup(test_pn)
        print(f'\nLookup "{test_pn}": {"FOUND" if result else "NOT FOUND"}')
    
    print('\n✓ utils/sw_library.py PASSED')
else:
    print('Skipped - no SW library uploaded')

In [ ]:
# Cell 10: Test utils/context_db.py
print('='*50)
print('TEST: utils/context_db.py')
print('='*50)

from ai_inspector.utils import ContextDatabase

context_db = ContextDatabase()
context_db.load(['.', SW_JSON_DIR, '/content'])

print(f'Part context entries: {context_db.part_context_count}')
print(f'Inspector requirements: {context_db.inspector_requirements_count}')

print('\n✓ utils/context_db.py PASSED')

In [ ]:
# Cell 11: Summary
print('='*50)
print('TEST SUMMARY')
print('='*50)
print('✓ config.py')
print('✓ models/ (PageArtifact, ResolvedPartIdentity, SwPartEntry, PageType, etc.)')
print('✓ utils/io.py (load_json_robust)')
print('✓ utils/pdf_render.py (render_pdf)' if pdf_path else '○ utils/pdf_render.py (skipped)')
print('✓ utils/sw_library.py (SwJsonLibrary)' if os.path.exists(SW_JSON_DIR) else '○ utils/sw_library.py (skipped)')
print('✓ utils/context_db.py (ContextDatabase)')
print('\nAll available modules working!')